In [1]:
import sys
import logging
import importlib
import itertools
from pathlib import Path

import numpy as np  # type: ignore
from matplotlib import pyplot as plt  # type: ignore

sys.path.append('../../src/')
import utils.figures  # type: ignore # noqa: E402

In [2]:
data_dir = Path('../../data/')

In [3]:
importlib.reload(utils.figures)
from utils.figures import load_all_eliminateds_dict  # noqa: E402
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)
eliminateds = load_all_eliminateds_dict(data_dir)
logging.basicConfig(level=logging.WARNING, stream=sys.stderr)

DEBUG:root:Loading positive-eliminated from ../../data/benchmark_results/all-full-flat-spencer-none.jsonl
DEBUG:root:Loading positive-eliminated from ../../data/benchmark_results/all-full-flat-spencer-cn.jsonl
DEBUG:root:Loading positive-eliminated from ../../data/benchmark_results/all-full-flat-spencer-in.jsonl


In [4]:
importlib.reload(utils.figures)
from utils.figures import load_memo_sizes_dict  # noqa: E402
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)
memo_sizes = load_memo_sizes_dict(data_dir)
logging.basicConfig(level=logging.WARNING, stream=sys.stderr)

DEBUG:root:Loading positive-memo_entry from ../../data/benchmark_results/all-full-thompson-spencer-none.jsonl
DEBUG:root:Loading negative-memo_entry from ../../data/benchmark_results/all-full-thompson-spencer-none.jsonl
DEBUG:root:Loading positive-memo_entry from ../../data/benchmark_results/all-full-thompson-spencer-cn.jsonl
DEBUG:root:Loading negative-memo_entry from ../../data/benchmark_results/all-full-thompson-spencer-cn.jsonl
DEBUG:root:Loading positive-memo_entry from ../../data/benchmark_results/all-full-thompson-spencer-in.jsonl
DEBUG:root:Loading negative-memo_entry from ../../data/benchmark_results/all-full-thompson-spencer-in.jsonl
DEBUG:root:Loading positive-memo_entry from ../../data/benchmark_results/all-full-flat-spencer-none.jsonl
DEBUG:root:Loading negative-memo_entry from ../../data/benchmark_results/all-full-flat-spencer-none.jsonl
DEBUG:root:Loading positive-memo_entry from ../../data/benchmark_results/all-full-flat-spencer-cn.jsonl
DEBUG:root:Loading negative-memo

In [5]:
# Histogram 2D
importlib.reload(plt)

logging.basicConfig(level=logging.WARNING, stream=sys.stderr, force=True)
options = itertools.product(['full', 'partial'], ['cn', 'in'])
threshold = 0

for matching_type, memo_scheme in options:
    input_types = ['positive', 'negative']
    xs = np.concatenate([
        memo_sizes[matching_type]['thompson'][memo_scheme][input_type]
        for input_type in input_types
    ])
    xlabel = "-".join([matching_type, 'thompson', memo_scheme])
    ys = np.concatenate([
        memo_sizes[matching_type]['flat'][memo_scheme][input_type]
        for input_type in input_types
    ])
    ylabel = "-".join([matching_type, 'flat', memo_scheme])
    zs = np.concatenate(
        [eliminateds[memo_scheme][input_type] for input_type in input_types])
    # Range excluding outliers
    total = xs + ys
    quartile_1 = np.quantile(total, 0.25)
    quartile_3 = np.quantile(total, 0.75)
    iqr = quartile_3 - quartile_1
    maximum = np.max(
        total, where=xs <= quartile_3 + 1.5 * iqr, initial=-np.inf)
    minimum = np.min(total, where=xs >= quartile_1 - 1.5 * iqr, initial=np.inf)
    x_space = np.geomspace(1, max(xs+ys), 50)
    y_space = np.geomspace(1, max(ys+ys), 50)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    h, _, _, cmap = axes[0].hist2d(
        xs[zs <= threshold], ys[zs <= threshold],
        bins=(x_space, y_space), norm='log'
    )
    axes[0].set_xscale('symlog')
    axes[0].set_yscale('symlog')
    axes[0].set_xlabel(xlabel + f" (elim. <= {threshold})")
    axes[0].set_ylabel(ylabel)
    axes[0].axis('square')

    vmax = np.max(h)

    axes[1].hist2d(xs[zs > threshold], ys[zs > threshold], bins=(
        x_space, y_space), norm='log', vmax=vmax)
    axes[1].set_xscale('symlog')
    axes[1].set_yscale('symlog')
    axes[1].set_xlabel(xlabel + f" (elim. > {threshold})")
    axes[1].axis('square')

    fig.subplots_adjust(right=0.8)
    axes[1].get_position()
    pos = axes[1].get_position()
    cbar_ax = fig.add_axes(
        [pos.x1 + 0.01, pos.y0 + 0.01, 0.01, pos.y1 - pos.y0 - 0.02])

    try:
        fig.colorbar(cmap, cax=cbar_ax)
        plt.subplots_adjust(wspace=0.3)
        output_dir = Path('./outputs/memo_sizes')
        output_dir.mkdir(parents=True, exist_ok=True)
        filename = "-".join([matching_type, memo_scheme]) + '.pdf'
        plt.savefig(output_dir / filename)
        plt.show()
    except Exception:
        continue

Error in callback <function _draw_all_if_interactive at 0x7f2754419a60> (for post_execute), with arguments args (),kwargs {}:


ValueError: Invalid vmin or vmax

ValueError: Invalid vmin or vmax

<Figure size 1200x600 with 3 Axes>

ValueError: Invalid vmin or vmax

<Figure size 1200x600 with 3 Axes>

ValueError: Invalid vmin or vmax

<Figure size 1200x600 with 3 Axes>

ValueError: Invalid vmin or vmax

<Figure size 1200x600 with 3 Axes>